<a href="https://colab.research.google.com/github/shaseebahmed/GenAI-Assignments/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Dataset (problem 1).csv to Dataset (problem 1) (1).csv


In [2]:
import pandas as pd
import io

file_path = list(uploaded.keys())[0]  # Get the uploaded file name
df = pd.read_csv(io.BytesIO(uploaded[file_path]))

# Display first few rows
df.head()


,Unnamed: 0,text,sentiment
0,0,text,0
1,1,rising cases of covid does not alarm me rising...,1
2,2,please vote for chicagoindiaresolution marking...,0
3,3,wishing all of you eidaladha hazrat ibrahim as...,1
4,4,daily coronavirus cases in india top for first...,1


In [3]:
!pip install sentence-transformers scikit-learn pandas torch


In [4]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [5]:
print(df.head())


   Unnamed: 0                                               text  sentiment
0           0                                               text          0
1           1  rising cases of covid does not alarm me rising...          1
2           2  please vote for chicagoindiaresolution marking...          0
3           3  wishing all of you eidaladha hazrat ibrahim as...          1
4           4  daily coronavirus cases in india top for first...          1


In [10]:
TEXT_COLUMN = "text"  # Replace with actual tweet text column name
LABEL_COLUMN = "sentiment"  # Replace with actual sentiment column (0 = Non-depressive, 1 = Depressive)


In [11]:
df.head()

,Unnamed: 0,text,sentiment
0,0,text,0
1,1,rising cases of covid does not alarm me rising...,1
2,2,please vote for chicagoindiaresolution marking...,0
3,3,wishing all of you eidaladha hazrat ibrahim as...,1
4,4,daily coronavirus cases in india top for first...,1


In [12]:
df = df.dropna(subset=[TEXT_COLUMN, LABEL_COLUMN])


In [13]:
df[LABEL_COLUMN] = df[LABEL_COLUMN].astype(int)


In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2").to(device)
print("Model Loaded!")


Model Loaded!


In [25]:
tweets = df[TEXT_COLUMN].tolist()
embeddings = model.encode(
    tweets, batch_size=2, show_progress_bar=True, convert_to_tensor=True
)


Batches:   0%|          | 0/67165 [00:00<?, ?it/s]

In [26]:
X = embeddings.cpu().numpy()
y = df[LABEL_COLUMN].values


In [27]:

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
# Train a simple classifier (Logistic Regression)
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [29]:
# Predictions
y_pred = clf.predict(X_test)


In [30]:
# Model Evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 0.7525
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.76      0.75     13118
           1       0.77      0.74      0.75     13748

    accuracy                           0.75     26866
   macro avg       0.75      0.75      0.75     26866
weighted avg       0.75      0.75      0.75     26866

